In [ ]:
import pandas as pd
from google.colab import drive
import json
from geopy.distance import geodesic
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
drive.mount('/content/drive')
BASE_PATH = '/content/drive/MyDrive/almrrc2021'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Loading all the package data

package_data = json.load(open(BASE_PATH + '/almrrc2021-data-training/model_build_inputs/package_data.json'))

flattened_package_data = []
for route_id, stops in package_data.items():
    for stop_id, packages in stops.items():
        for package_id, details in packages.items():
            record = {
                "route_id": route_id,
                "stop_id": stop_id,
                "package_id": package_id,
                "scan_status": details["scan_status"] if "scan_status" in details else None, # note model_apply_inputs does not have scan_status, model_build_inputs does
                "start_time_utc": details["time_window"]["start_time_utc"],
                "end_time_utc": details["time_window"]["end_time_utc"],
                "planned_service_time_seconds": details["planned_service_time_seconds"],
                "depth_cm": details["dimensions"]["depth_cm"],
                "height_cm": details["dimensions"]["height_cm"],
                "width_cm": details["dimensions"]["width_cm"],
            }
            flattened_package_data.append(record)

package_data_df = pd.DataFrame(flattened_package_data)

package_data_df.head()

,route_id,stop_id,package_id,scan_status,start_time_utc,end_time_utc,planned_service_time_seconds,depth_cm,height_cm,width_cm
0,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,AD,PackageID_9d7fdd03-f2cf-4c6f-9128-028258fc09ea,DELIVERED,NaN,NaN,59.3,25.4,7.6,17.8
1,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,AD,PackageID_5541e679-b7bd-4992-b288-e862f6c84ae7,DELIVERED,2018-07-27 16:00:00,2018-07-28 00:00:00,59.3,25.4,12.7,17.8
2,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,AD,PackageID_84d0295b-1adb-4a33-a65e-f7d6247c7a07,DELIVERED,NaN,NaN,59.3,39.4,7.6,31.8
3,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,AF,PackageID_15c6a204-ec5f-4ced-9c3d-472316cc7759,DELIVERED,2018-07-27 16:00:00,2018-07-28 00:00:00,27.0,30.0,3.0,27.4
4,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,AG,PackageID_3b28f781-242e-416e-9575-84c7188b8208,DELIVERED,NaN,NaN,45.0,25.4,12.7,17.8


In [ ]:
# Loading all the route data

route_data = json.load(open(BASE_PATH + '/almrrc2021-data-training/model_build_inputs/route_data.json'))

flattened_route_data = []

for route_id, info in route_data.items():
    for stop_id, stop_details in info['stops'].items():
        flattened_route_data.append({
            "route_id": route_id,
            "station_code": info['station_code'],
            "date": info['date_YYYY_MM_DD'],
            "departure_time_utc": info['departure_time_utc'],
            "executor_capacity_cm3": info['executor_capacity_cm3'],
            "route_score": info['route_score'],
            "stop_id": stop_id,
            "lat": stop_details['lat'],
            "lng": stop_details['lng'],
            "type": stop_details['type'],
            "zone_id": stop_details['zone_id']
        })

route_data_df = pd.DataFrame(flattened_route_data)

route_data_df.head()

,route_id,station_code,date,departure_time_utc,executor_capacity_cm3,route_score,stop_id,lat,lng,type,zone_id
0,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,AD,34.099611,-118.283062,Dropoff,P-12.3C
1,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,AF,34.101587,-118.291125,Dropoff,A-1.2D
2,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,AG,34.089727,-118.285530,Dropoff,A-2.1A
3,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,BA,34.096132,-118.292869,Dropoff,A-1.2C
4,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,BE,34.098482,-118.286243,Dropoff,P-13.3B


In [ ]:
# Loading all the travel times

vehicle_travel_times = json.load(open(BASE_PATH + '/almrrc2021-data-training/model_build_inputs/travel_times.json'))

**To create a solution to the VRPD for we need the following information for a route:**

\\

We already have:

Package Data
*   Package ID
*   Dimensions (length, width, height)
*   Service time seconds

Stop Data
*   Stop ID
*   Latitude and Longitude
*   Zone ID
*   List of Package ID(s) to be delivered
*   Depot bool

Travel Times:
*   Vehicle transit time to every other stop

\\
We will need to calculate
*   Drone bool for each stop ((if a drone can cover this stop ie. one small package)
* Drone transit time to every other stop

\\

We can begin by collecting all the data we want for a desired route





In [ ]:
def get_data_for_route(route_id):
    package_data = package_data_df[package_data_df['route_id'] == route_id]
    route_data = route_data_df[route_data_df['route_id'] == route_id]
    travel_times_route = vehicle_travel_times[route_id]

    return package_data.copy(), route_data.copy(), travel_times_route.copy()

In [ ]:
# Given route data, it will return a dictionary
def calculate_drone_travel_time(route_data):
  drone_travel_time = {}

  for _, base_row in route_data.iterrows():
    drone_travel_time[base_row['stop_id']] = {}

    # calculate travel time to every other stop in the route
    for _, other_row in route_data.iterrows():
      if base_row['stop_id'] != other_row['stop_id']:
        base_location = (base_row['lat'], base_row['lng'])
        other_location = (other_row['lat'], other_row['lng'])

        distance = geodesic(base_location, other_location).meters

        # Assuming the drone will travel 7 m/s TODO: CONSTANT
        drone_travel_time[base_row['stop_id']][other_row['stop_id']] = distance / 7.0
        # drone_travel_time[base_row['stop_id']][other_row['stop_id']] += 10 # 5 second rise, 5 second drop

      else:
        drone_travel_time[base_row['stop_id']][other_row['stop_id']] = 0

  return drone_travel_time

In [ ]:
package_data_route_df, route_data_route_df, vehicle_travel_times_route = get_data_for_route("RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77")

In [ ]:
drone_travel_times_route = calculate_drone_travel_time(route_data_route_df)

In [ ]:
# Determining if each package can be delivered by a drone
package_data_route_df['drone_possible'] = (package_data_route_df['depth_cm'] < 30) & (package_data_route_df['height_cm'] < 25) & (package_data_route_df['width_cm'] < 30) # TODO: be indifferent on depth and width

In [ ]:
package_info_agg = package_data_route_df.groupby('stop_id').apply(
    lambda x: pd.Series({
        'packages': x[['package_id', 'depth_cm', 'height_cm', 'width_cm']].to_dict('records'),
        'num_packages': len(x),
        'drone_possible': (len(x) == 1) and (x['drone_possible'].iloc[0] == True),
        'service_time_seconds': x['planned_service_time_seconds'].mean() # Not sure if this should be sum or average. https://github.com/MIT-CAVE/rc-cli/blob/main/templates/data_structures.md
    })
).reset_index()

In [ ]:
route_data_route_df = pd.merge(route_data_route_df, package_info_agg, how='left', on='stop_id')

In [ ]:
route_data_route_df.head()

,route_id,station_code,date,departure_time_utc,executor_capacity_cm3,route_score,stop_id,lat,lng,type,zone_id,packages,num_packages,drone_possible,service_time_seconds
0,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,AD,34.099611,-118.283062,Dropoff,P-12.3C,[{'package_id': 'PackageID_9d7fdd03-f2cf-4c6f-...,3.0,False,59.3
1,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,AF,34.101587,-118.291125,Dropoff,A-1.2D,[{'package_id': 'PackageID_15c6a204-ec5f-4ced-...,1.0,False,27.0
2,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,AG,34.089727,-118.285530,Dropoff,A-2.1A,[{'package_id': 'PackageID_3b28f781-242e-416e-...,2.0,False,45.0
3,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,BA,34.096132,-118.292869,Dropoff,A-1.2C,[{'package_id': 'PackageID_a18e36e0-6b5a-45b7-...,1.0,True,38.0
4,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,BE,34.098482,-118.286243,Dropoff,P-13.3B,[{'package_id': 'PackageID_22c8f630-8ab9-40d5-...,4.0,False,41.8


In [ ]:
# These are all the stops which could be serviced by a drone
route_data_route_df[route_data_route_df['type'] == 'Station']

,route_id,station_code,date,departure_time_utc,executor_capacity_cm3,route_score,stop_id,lat,lng,type,zone_id,packages,num_packages,drone_possible,service_time_seconds
103,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,VE,34.007369,-118.143927,Station,NaN,NaN,NaN,NaN,NaN


## Naive Solver
This solver will just go to the next closest stop within the Zone, and then the next closest stop in a different zone. After that, the path is augmented with drone deliveries

In [ ]:
# Event Types

# Both
LOAD_START = 'LOAD_START'
LOAD_END = 'LOAD_END'

# Vehicle
ARRIVE = 'ARRIVE'
SERVICE_START = 'SERVICE_START'
SERVICE_END = 'SERVICE_END'
DEPART = 'DEPART'

# Drone
LAUNCH = 'LAUNCH'
DELIVER = 'DELIVER'
LAND = 'LAND'

In [ ]:
class Event:
  def __init__(self, event_type, stop_id, time, data): # data holds extra information, like coordinates
    self.event_type = event_type
    self.stop_id = stop_id
    self.time = time
    self.data = data

  def __str__(self):
    return f"{self.event_type} {self.stop_id} {round(self.time, 1)} {self.data}"

In [ ]:
class VRPDSolver:
  def __init__(self, package_data, route_data, vehicle_travel_times, drone_travel_times):
    self.package_data = package_data
    self.route_data = route_data
    self.vehicle_travel_times = vehicle_travel_times
    self.drone_travel_times = drone_travel_times
    self.station = self.route_data[self.route_data['type'] == 'Station'].iloc[0]

    self.vehicle_path = []
    self.vehicle_events = []
    self.drone_path = []
    self.drone_events = []

    self.remaining_stops = set(self.route_data['stop_id'].unique())
    self.completed_stops = set()


    self.cur_stop = None
    self.cur_zone = None
    self.cur_coords = None
    self.cur_time = 0

  # Get closest stop
  def get_closest_stop(self):

    possible_zones = None
    if self.cur_zone and len(self.cur_zone) > 0:
      # Get all possible zones
      possible_zones = self.route_data[self.route_data['zone_id'] == self.cur_zone]
      # Remove any zones that have been visited
      possible_zones = possible_zones[~possible_zones['stop_id'].isin(self.completed_stops)]

    # If there are more stops left in the zone, we find the closest one using it's stop_id and vehicle_travel_times
    best_next_stop = None
    cur_min_time = float('inf')

    # We find the next closest stop in the zone
    if possible_zones is not None and not possible_zones.empty:
      for _, row in possible_zones.iterrows():
        if self.vehicle_travel_times[self.cur_stop][row['stop_id']] < cur_min_time and row['stop_id'] != self.cur_stop:
          cur_min_time = self.vehicle_travel_times[self.cur_stop][row['stop_id']]
          best_next_stop = row['stop_id']
    else:
      # We find the next closest stop that has not been visited
      for _, row in self.route_data.iterrows(): # Can be optimized by iterating through remaining_stops
        if row['stop_id'] not in self.completed_stops:
          if self.vehicle_travel_times[self.cur_stop][row['stop_id']] < cur_min_time and row['stop_id'] != self.cur_stop:
            cur_min_time = self.vehicle_travel_times[self.cur_stop][row['stop_id']]
            best_next_stop = row['stop_id']


    return best_next_stop

  def naive_vrp(self):
    # The first step is departing the station
    self.cur_stop = self.station['stop_id']
    self.cur_coords = {'cordinates' : {'lat' : self.station['lat'], 'lng' : self.station['lng']}}

    # We leave the depot station
    first_event = Event(DEPART, self.cur_stop, self.cur_time, self.cur_coords)
    self.vehicle_events.append(first_event)
    self.vehicle_path.append(self.cur_stop)

    while len(self.remaining_stops) > 0:
      # We find the closest stop
      next_stop = self.get_closest_stop()
      # Travel there
      self.cur_time += self.vehicle_travel_times[self.cur_stop][next_stop]
      # Arrive
      self.cur_stop = next_stop
      stop = self.route_data[self.route_data['stop_id'] == self.cur_stop].iloc[0]
      self.cur_zone = stop['zone_id']
      self.cur_coords = {'cordinates' : {'lat' : stop['lat'], 'lng' : stop['lng']}}
      arrive_event = Event(ARRIVE, self.cur_stop, self.cur_time, self.cur_coords)
      self.vehicle_events.append(arrive_event)
      self.vehicle_path.append(self.cur_stop)

      # # Edge case: If we arrive at the station we can break before delivering packages
      if self.cur_stop == self.station['stop_id']:
        break

      # Driver delivers package
      service_start_event = Event(SERVICE_START, self.cur_stop, self.cur_time, self.cur_coords)
      self.vehicle_events.append(service_start_event)
      self.cur_time += stop['service_time_seconds']
      service_end_event = Event(SERVICE_END, self.cur_stop, self.cur_time, self.cur_coords)
      self.vehicle_events.append(service_end_event)
      self.completed_stops.add(self.cur_stop)
      self.remaining_stops.remove(self.cur_stop)
      # Driver departs
      depart_event = Event(DEPART, self.cur_stop, self.cur_time, self.cur_coords)
      self.vehicle_events.append(depart_event)

  def print_vehicle_path(self):
      print(self.vehicle_path)

  def print_vehicle_events(self):
    for event in self.vehicle_events:
        print(event)




In [ ]:
test = VRPDSolver(package_data_route_df, route_data_route_df, vehicle_travel_times_route, drone_travel_times_route)
test.naive_vrp()

In [ ]:
test.print_vehicle_events()

DEPART VE 0 {'cordinates': {'lat': 34.007369, 'lng': -118.143927}}
ARRIVE TG 1509.7 {'cordinates': {'lat': 34.088467, 'lng': -118.284521}}
SERVICE_START TG 1509.7 {'cordinates': {'lat': 34.088467, 'lng': -118.284521}}
SERVICE_END TG 1536.1 {'cordinates': {'lat': 34.088467, 'lng': -118.284521}}
DEPART TG 1536.1 {'cordinates': {'lat': 34.088467, 'lng': -118.284521}}
ARRIVE GP 1561.5 {'cordinates': {'lat': 34.088709, 'lng': -118.284839}}
SERVICE_START GP 1561.5 {'cordinates': {'lat': 34.088709, 'lng': -118.284839}}
SERVICE_END GP 1595.5 {'cordinates': {'lat': 34.088709, 'lng': -118.284839}}
DEPART GP 1595.5 {'cordinates': {'lat': 34.088709, 'lng': -118.284839}}
ARRIVE HT 1632.9 {'cordinates': {'lat': 34.088717, 'lng': -118.286484}}
SERVICE_START HT 1632.9 {'cordinates': {'lat': 34.088717, 'lng': -118.286484}}
SERVICE_END HT 1705.4 {'cordinates': {'lat': 34.088717, 'lng': -118.286484}}
DEPART HT 1705.4 {'cordinates': {'lat': 34.088717, 'lng': -118.286484}}
ARRIVE AG 1767.6 {'cordinates': {

In [ ]:
test.print_vehicle_path()

['VE', 'TG', 'GP', 'HT', 'AG', 'QM', 'BY', 'SF', 'JH', 'LB', 'FH', 'DL', 'TK', 'CW', 'VW', 'RA', 'MQ', 'BP', 'MA', 'IM', 'EY', 'EH', 'SQ', 'XD', 'ZP', 'FF', 'HG', 'MO', 'YN', 'CK', 'QE', 'LK', 'LY', 'PS', 'PJ', 'GW', 'RY', 'FY', 'HO', 'YR', 'WS', 'NM', 'GB', 'US', 'IA', 'EO', 'UI', 'KG', 'BG', 'CO', 'KM', 'MW', 'KA', 'BT', 'CA', 'TC', 'GN', 'KJ', 'LD', 'NE', 'AF', 'PX', 'YY', 'SD', 'UU', 'WJ', 'AD', 'EC', 'QO', 'UN', 'YJ', 'TY', 'EX', 'RG', 'VC', 'BZ', 'ZU', 'KU', 'NU', 'GS', 'YE', 'PT', 'KN', 'XB', 'IJ', 'SC', 'GU', 'NR', 'KP', 'IW', 'TH', 'UW', 'BA', 'YH', 'DQ', 'CP', 'DN', 'UJ', 'QX', 'BE', 'PB', 'HR', 'ZB', 'IP', 'SI', 'CM', 'HB', 'HW', 'JM', 'TQ', 'MR', 'CG', 'LG', 'UR', 'ZE', 'VA', 'NL', 'DJ', 'HN', 'VE']


In [ ]:
route_data_ordered = pd.concat([route_data_route_df[route_data_route_df['stop_id'] == stop_id] for stop_id in test.vehicle_path]).reset_index(drop=True)
route_data_ordered.head()

,route_id,station_code,date,departure_time_utc,executor_capacity_cm3,route_score,stop_id,lat,lng,type,zone_id,packages,num_packages,drone_possible,service_time_seconds
0,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,VE,34.007369,-118.143927,Station,NaN,NaN,NaN,NaN,NaN
1,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,TG,34.088467,-118.284521,Dropoff,A-2.2A,[{'package_id': 'PackageID_fa1e5684-a42e-40d0-...,7.0,False,26.4
2,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,GP,34.088709,-118.284839,Dropoff,A-2.2A,[{'package_id': 'PackageID_99fdaa4b-9b64-4b6d-...,1.0,False,34.0
3,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,HT,34.088717,-118.286484,Dropoff,A-2.2A,[{'package_id': 'PackageID_581892a8-0e1c-46b1-...,2.0,False,72.5
4,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,AG,34.089727,-118.285530,Dropoff,A-2.1A,[{'package_id': 'PackageID_3b28f781-242e-416e-...,2.0,False,45.0


In [ ]:
fig = go.Figure(go.Scattermapbox(
    lat=route_data_ordered['lat'],
    lon=route_data_ordered['lng'],
    mode='lines',
    line=dict(width=2, color='blue'),  # Customize line color and width here
    hoverinfo='none'
))

# Add scatter points for each unique zone_id, with custom hover text
for zone_id in route_data_ordered['zone_id'].unique():
    df_sub = route_data_ordered[route_data_ordered['zone_id'] == zone_id]

    # Creating custom hover text
    hover_text = df_sub.apply(lambda row: f"Sequence: {row.name}<br> Stop ID: {row['stop_id']}<br>Zone ID: {row['zone_id']}<br>Num Packages: {row['num_packages']} <br> Service Time: {row['service_time_seconds']}", axis=1)


    fig.add_trace(go.Scattermapbox(
        lat=df_sub['lat'],
        lon=df_sub['lng'],
        mode='markers',
        marker=go.scattermapbox.Marker(size=15),
        name=str(zone_id),
        text=hover_text,
        hoverinfo='text'
    ))

# Update the layout to use OpenStreetMap style and adjust other layout properties
fig.update_layout(

    height=600,
    mapbox=dict(
        style="open-street-map",
        zoom=10,  # Adjust zoom level here
        center=dict(lat=route_data_ordered['lat'].mean(), lon=route_data_ordered['lng'].mean())
    ),
    showlegend=False  # Set to True if you want to show legend
)
fig.update_layout(margin=dict(l=10, r=10, t=10, b=10))


# Show the figure
fig.show()